**Задание1**

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

***Решение***

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import json

In [3]:
keywords = ['python', 'парсинг']

post_time = []
title = []
article_url = []
url_list= []

for url in BeautifulSoup(requests.get('https://habr.com/ru/all/').text, 'html.parser').find\
('div', class_='content_left js-content_left').find_all('a', class_='post__title_link'):
    url_list.append(url.get('href'))

for i in url_list: 
    soup = BeautifulSoup(requests.get(i).text, 'html.parser') 

    for keyword in keywords: 
        x = soup.find('div', class_='post__text post__text-html post__text_v1').get_text()

        if keyword in x.lower(): 
            post_time.append(soup.find('span', class_='post__time').get_text()) 
            title.append(soup.find('span', class_='post__title-text').get_text()) 
            article_url.append(url_list)

articles_data = pd.DataFrame({'Дата': post_time,'Заголовок': title,'Ссылка': article_url})
articles_data

,Дата,Заголовок,Ссылка
0,сегодня в 22:05,Интервью из мира хостинга: Boodet.online,"[https://habr.com/ru/post/515538/, https://hab..."
1,сегодня в 18:02,Разворачиваем модель машинного обучения с Dock...,"[https://habr.com/ru/post/515538/, https://hab..."
2,сегодня в 17:39,Как писать аккуратные конвейеры для машинного ...,"[https://habr.com/ru/post/515538/, https://hab..."


**Задание2**

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. 

Список email-ов задаем переменной в начале кода:

EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

***Решение***

In [10]:
emails = ['xxx@x.ru', 'yyy@y.com']
leaks = []
url_='https://digibody.avast.com/v1/web/leaks'

def leak_info(email, leak):
    try:
        res = {
            'почта': email,
            'дата утечки': dt.datetime.fromtimestamp(leak['leak_date']/1000),
            'источник утечки': leak['leak_info']['title'],
            'описание утечки': leak['leak_info']['description']
        }
        return res
    except:
        log_exception('Ошибка: {}'.format(leak))
    return None

def check_email_leaks(email):
    req_params = {'email': email}
    res = requests.post(url_, data=json.dumps(req_params))
    if res.status_code != 200:
        raise Exception('Неверный код: {}'.format(res.status_code))
    resp = json.loads(res.text)
    if resp['status'] != 'ok':
        raise Exception('Текущий статус "{}" != "ok"'.format(resp['status']))
        
    res = [ leak_info(email, leak) for leak in resp['value'] ]
    return list(filter(None.__ne__, res))

for i in emails:
    try:
        check_res = check_email_leaks(i)      
        leaks = leaks + check_res
    except:
        log_exception(f'Не удалось обработать электронную почту "{i}"')

df = pd.DataFrame(leaks)
df

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2017-12-22 03:00:00,Combolist of 1.4 Billion Credentials,The proliferation of stolen or leaked database...
1,xxx@x.ru,2019-02-06 03:00:00,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
2,xxx@x.ru,2017-01-31 03:00:00,CD Projekt Red,"In March 2016, CDProjektRed.com.com's forum da..."
3,xxx@x.ru,2017-02-14 03:00:00,Parapa,"In July and August 2016, two criminals execute..."
4,xxx@x.ru,2017-03-08 03:00:00,Sensitive Source,This source has been marked as sensitive due t...
5,xxx@x.ru,2019-02-06 03:00:00,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
6,xxx@x.ru,2019-02-06 03:00:00,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."
7,xxx@x.ru,2017-02-14 03:00:00,CFire Mail,"In July and August of 2016, two criminals carr..."
8,xxx@x.ru,2016-10-21 03:00:00,Adobe Systems,"In October of 2013, criminals penetrated Adobe..."
9,xxx@x.ru,2019-01-29 03:00:00,Collection #2 Combo List,"On January 7, 2019, an online user named Sanix..."
